In [1]:
import tensorflow as tf
import gensim


In [2]:
import numpy as np
import pandas as pd


In [24]:
import pickle
import numpy as np
import spacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [33]:
VALIDATION_SIZE = 500


In [3]:
path_to_data = "/Users/anne/repos/AEM/data/dataset_vermeer.pkl"


In [4]:
df = pd.read_pickle(path_to_data)


In [25]:
def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = tf.keras.utils.to_categorical(encoded_Y)
    return dummy_y

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], encodeY(df['topic'].map(int)), test_size = 0.2)
del df


In [28]:
vectorizer = CountVectorizer(min_df=5, max_df=.9)
X_train_onehot = vectorizer.fit_transform(X_train)


In [29]:
numberoflabels = len(set((str(e) for e in y_train.tolist())))
numberoflabels


4

In [30]:
model = Sequential()
model.add(Dense (units = 500, activation = 'relu', input_dim = len(vectorizer.get_feature_names())))
#model.add(Dense(units = 400, activation = 'sigmoid'))
#model.add(Dense(units = 300, activation = 'sigmoid'))
model.add(Dense(units = 200, activation = 'sigmoid'))
# model.add(Dense(units = 100, activation = 'sigmoid'))
model.add(Dense(units = numberoflabels, activation = 'sigmoid'))

W0730 10:22:27.863028 4709397952 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [31]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               9598500   
_________________________________________________________________
dense_1 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 804       
Total params: 9,699,504
Trainable params: 9,699,504
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(X_train_onehot[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=1, verbose=1,
          validation_data=(X_train_onehot[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

W0730 10:23:17.639453 4709397952 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2288 samples, validate on 500 samples
2288/2288 [==============================] - 7s 3ms/sample - loss: 0.7355 - acc: 0.7295 - val_loss: 0.4637 - val_acc: 0.8380


In [35]:
scores = model.evaluate(vectorizer.transform(X_test), y_test, verbose = True)
print("Accuracy: {}".format(scores[1]))

698/698 [==============================] - 0s 650us/sample - loss: 0.5252 - acc: 0.8181
Accuracy: 0.8180515766143799


## Convolutional Network

In [37]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train]

In [38]:
# Compute the max lenght of a text
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
print(X_train_sequences[0])

MAX_SEQ_LENGTH= 19990
[19196 19196 19196 ... 14704  9208 16675]


In [40]:
model = Sequential()
model.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGTH))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0730 10:24:44.891540 4709397952 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19990, 64)         1228608   
_________________________________________________________________
conv1d (Conv1D)              (None, 19986, 64)         20544     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3997, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 255808)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16371776  
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 260       
Total params: 17,621,188
Trainable params: 17,621,188
Non-trainable params: 0
__________________________________________

In [41]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=1, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


Train on 2288 samples, validate on 500 samples
2288/2288 [==============================] - 142s 62ms/sample - loss: 1.1456 - acc: 0.5586 - val_loss: 0.9357 - val_acc: 0.6340


In [43]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)


In [44]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])


698/698 [==============================] - 10s 14ms/sample - loss: 1.0521 - acc: 0.5458
Accuracy: 0.54584527


### Embeddings

In [47]:
em = '/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15.txt'
em_2 = '/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15'
cow = '/Users/anne/repos/embedding_models/test/cow-320.txt'


In [96]:
wauw = '/Users/anne/repos/embedding_models/AEM_corpus/w2v_model_nr_5_window_10_size_300_negsample_5.txt'


In [ ]:
embeddings = np.load(open(cow, mode='rb'))


In [101]:
# dit hier
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        

In [102]:
embedding_matrix


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        

In [79]:
embeddings.wv[embeddings.wv.index2word[5]]


array([ 1.0844518e-01, -2.4136243e+00, -4.5471078e-01,  7.2799361e-01,
       -2.8631926e+00,  7.5468111e-01, -4.1612883e+00,  4.3661171e-01,
       -2.3788567e+00, -2.0556877e+00,  7.6177251e-01,  8.6660033e-01,
       -3.9178357e-01,  4.2466097e-02, -1.0125976e+00,  2.0925124e+00,
       -7.4650484e-01, -1.1795570e+00,  2.6217375e+00, -2.2584314e+00,
       -5.3062093e-01, -4.0050807e+00, -3.4212801e-01, -2.0599627e+00,
       -2.1454351e+00,  3.3439126e+00,  3.0396247e+00,  1.8601815e+00,
       -3.3948326e-01, -3.9520605e+00,  9.9820930e-01, -4.5853540e-01,
       -5.1464778e-01, -1.3669431e-01, -7.0555133e-01, -2.6589320e+00,
       -2.6402314e+00,  9.0452433e-01,  2.4894533e+00,  5.3115636e-01,
       -3.0481358e+00, -3.6102155e-01, -1.9276475e+00,  7.0000744e-01,
        7.5999743e-01, -1.1167306e+00,  1.7887955e+00,  2.8596738e-01,
       -4.5045018e-01,  1.6437404e+00,  4.3696740e-01,  1.9551530e-02,
        5.6050968e+00, -2.2517145e+00,  4.3023291e-01, -7.4326649e-02,
      

In [69]:
embeddings.vector_size


300

In [103]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        

In [104]:
embedding_matrix


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
dimensions = embeddings.vector_size


300

In [105]:
embedding_layer = Embedding(N_FEATURES + 1,
                            dimensions,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)


In [106]:
embeddings_index = {}
with open(wauw) as f:
    numberofwordvectors, dimensions = [int(e) for e in next(f).split()]
    for line in f:
        values = line.split()
        embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
        
       # word = values[0]
       # coefs = np.asarray(values[1:], dtype='float32')
      #  embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
print('Should be {} vectors with {} dimensions'.format(numberofwordvectors, dimensions))


KeyboardInterrupt: 

In [ ]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

# load embedding from file
raw_embedding = load_embedding(wauw)
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, t.word_index)


In [ ]:
embedding_vectors = get_weight_matrix(raw_embedding, t.word_index)


In [98]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

def get_weight_matrix(embedding, vocab):
# total vocabulary size plus 0 for unknown words
vocab_size = len(vocab) + 1
# define weight matrix dimensions with all 0
weight_matrix = zeros((vocab_size, 100))
# step vocab, store vectors using the Tokenizer's integer mapping
for word, i in vocab.items():
    weight_matrix[i] = embedding.get(word)
return weight_matrix

        

In [ ]:
embedding_matrix.sum()


In [93]:
embedding_layer = Embedding(N_FEATURES + 1,
                            dimensions,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)

In [94]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 19990, 320)        6143040   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 19986, 64)         102464    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3997, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 255808)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                16371776  
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 260       
Total params: 22,617,540
Trainable params: 16,474,500
Non-trainable params: 6,143,040
__________________________________

In [95]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=1, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


Train on 2288 samples, validate on 500 samples
2288/2288 [==============================] - 464s 203ms/sample - loss: 1.0610 - acc: 0.5677 - val_loss: 0.7920 - val_acc: 0.7320


In [85]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)


In [86]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])


698/698 [==============================] - 33s 47ms/sample - loss: 1.4789 - acc: 0.2034
Accuracy: 0.2034384


### NEW TRY

In [115]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 300))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix


In [112]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)


In [113]:
 # define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1


In [116]:
# load embedding from file
raw_embedding = load_embedding(wauw)
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)


KeyboardInterrupt: 

In [ ]:
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_vectors], input_length=max_length, trainable=False)


In [6]:
t = tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(data)
vocab_size = len(t.word_index) + 1


In [ ]:
# Compute the max lenght of a text
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
print(X_train_sequences[0])


In [7]:
encoded_docs = t.texts_to_sequences(data)
print(encoded_docs)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


In [11]:
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 300))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [12]:
raw_embedding = load_embedding(em)

NameError: name 'em' is not defined

In [36]:
embedding_vectors = get_weight_matrix(raw_embedding, t.word_index)
embedding_vectors

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [39]:
model = tf.keras.Sequential()
e = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_vectors], input_length=4, trainable=False)
model.add(e)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

In [168]:

model =  tf.keras.Sequential()
e = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_vectors], input_length=4, trainable=False)
model.add(e)
#model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
#model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 4, 300)            31257600  
_________________________________________________________________
flatten_8 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                76864     
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 260       
Total params: 31,334,724
Trainable params: 77,124
Non-trainable params: 31,257,600
_________________________________________________________________
None


In [169]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


ValueError: Error when checking input: expected embedding_16_input to have shape (4,) but got array with shape (34549,)

In [95]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


ValueError: Error when checking input: expected embedding_9_input to have shape (4,) but got array with shape (34549,)

In [11]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes


X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train]

NameError: name 'vectorizer' is not defined

In [91]:
# Compute the max lenght of a text
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
print(X_train_sequences[0])

MAX_SEQ_LENGTH= 34549
[19530 19530 19530 ... 10759  9339 16963]


In [92]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


ValueError: Error when checking input: expected embedding_8_input to have shape (4,) but got array with shape (34549,)

In [65]:
model.fit(padded_docs, labels, epochs=50, verbose=0)
model.evaluate(padded_docs, labels)


3486/3486 [==============================] - 0s 44us/sample - loss: nan - acc: 0.0000e+00


[nan, 0.0]

In [66]:
VALIDATION_SIZE = 500


In [96]:
model.fit(padded_docs[:-VALIDATION_SIZE], labels[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(padded_docs[-VALIDATION_SIZE:], labels[-VALIDATION_SIZE:]))


Train on 2986 samples, validate on 500 samples
Epoch 1/3
2986/2986 [==============================] - 0s 109us/sample - loss: nan - acc: 3.3490e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
2986/2986 [==============================] - 0s 62us/sample - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
2986/2986 [==============================] - 0s 61us/sample - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00


In [12]:
def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = tf.keras.utils.to_categorical(encoded_Y)
    return dummy_y

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['text'], encodeY(df['topic'].map(int)), test_size = 0.2)


In [14]:
vectorizer = CountVectorizer(min_df=5, max_df=.9)
X_train_onehot = vectorizer.fit_transform(X_train)


In [15]:
numberoflabels = len(set((str(e) for e in y_train.tolist())))
numberoflabels


4

In [87]:
model.fit(X_train_onehot[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=1,
          validation_data=(X_train_onehot[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


ValueError: Error when checking input: expected embedding_7_input to have shape (4,) but got array with shape (19530,)

In [97]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])
]

NameError: name 'X_test_sequences' is not defined

In [119]:
from gensim.models import word2vec
model = gensim.models.Word2Vec.load('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018')

model.wv.save_word2vec_format('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018.txt', binary=False)

In [ ]:
model.wv.save_word2vec_format('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018.txt', binary=False)

In [128]:
model = KeyedVectors.load('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018')


In [116]:
from gensim.models import KeyedVectors

In [140]:
model = KeyedVectors.load('/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15')

In [16]:
em = '/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15.txt'
#em = '/Users/anne/repos/embedding_models/test/w2v_model_nr_0_window_5_size_100_negsample_5'
cow = '/Users/anne/repos/embedding_models/test/cow-320.txt'

In [ ]:
w2v = gensim.models.Word2Vec.load("/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15")
vocab = w2v.wv.vocab    
t = Tokenizer()

vocab_size = len(all_words) + 1
t.fit_on_texts(all_words)

def get_weight_matrix():
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, w2v.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for i in range(len(all_words)):
        weight_matrix[i + 1] = w2v[all_words[i]]
    return weight_matrix

embedding_vectors = get_weight_matrix()
emb_layer = Embedding(vocab_size, output_dim=w2v.vector_size, weights=[embedding_vectors], input_length=FIXED_LENGTH, trainable=False)

In [ ]:
embeddings_index = {}
f = open(em)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [17]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train]

In [18]:
embeddings_index = {}
with open(em) as f:
    numberofwordvectors, dimensions = [int(e) for e in next(f).split()]
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
print('Should be {} vectors with {} dimensions'.format(numberofwordvectors, dimensions))

Found 1055100 word vectors.
Should be 1055100 vectors with 300 dimensions


In [184]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 300))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [22]:
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
print(X_train_sequences[0])

MAX_SEQ_LENGTH= 25610
[19080 19080 19080 ...  8563  3691  7062]


In [24]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector      

In [25]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [203]:
counter

19017

In [201]:
counter

18825

In [189]:
embedding_layer = tf.keras.layers.Embedding(N_FEATURES + 1,
                            dimensions,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)

In [190]:
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)


MAX_SEQ_LENGTH= 34549


In [191]:
model =  tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

ValueError: Layer weight shape (19531, 300) not compatible with provided weight shape (104192, 300)

In [148]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))


Train on 2288 samples, validate on 500 samples
Epoch 1/3
2288/2288 [==============================] - 534s 233ms/sample - loss: 1.6418 - acc: 0.2614 - val_loss: 1.4274 - val_acc: 0.2180
Epoch 2/3
2288/2288 [==============================] - 502s 219ms/sample - loss: 1.4011 - acc: 0.1980 - val_loss: 1.4135 - val_acc: 0.1820
Epoch 3/3
2288/2288 [==============================] - 683s 299ms/sample - loss: 1.3946 - acc: 0.1888 - val_loss: 1.3983 - val_acc: 0.1680


In [149]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)

In [150]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

698/698 [==============================] - 54s 77ms/sample - loss: 1.3743 - acc: 0.1547
Accuracy: 0.15472779


In [114]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

698/698 [==============================] - 60s 85ms/sample - loss: 0.6765 - acc: 0.7650
Accuracy: 0.76504296
